In [1]:
import torch
from torch import nn
from torch.nn import functional as F

from pathlib import Path

In [2]:
data_path = Path("../data/tiny_shakespear.txt")

with data_path.open() as f:
    text = f.read()

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

"".join(chars), vocab_size

("\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz", 65)

In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l]) 

print(e := encode("Hello There!"))
print(decode(e))

[20, 43, 50, 50, 53, 1, 32, 46, 43, 56, 43, 2]
Hello There!


In [5]:
# Train and test splits
data = torch.tensor(encode(text))
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]

train_data

tensor([18, 47, 56,  ..., 43, 56, 43])

In [6]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 20 # what is the maximum context length for predictions?
epochs = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 200
# ------------

In [7]:
def get_batch(split):
    # random generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # Average attention
    x, y = x.to(device), y.to(device)
    
    return x, y

In [8]:
x, y = get_batch("train")
x[0], y[0]

(tensor([52, 49, 57,  1, 58, 53, 53,  1, 51, 59, 41, 46,  8,  0,  0, 30, 27, 25,
         17, 27]),
 tensor([49, 57,  1, 58, 53, 53,  1, 51, 59, 41, 46,  8,  0,  0, 30, 27, 25, 17,
         27, 10]))

In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        tokens = self.token_embedding_table(idx) # (B,T,C)
        logits = self.lm_head(tokens)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        model.eval()
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    
    return out

In [11]:
model = BigramLanguageModel().to(device)

In [12]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 200)
  (lm_head): Linear(in_features=200, out_features=65, bias=True)
)

In [13]:
# Expected loss
-torch.log(torch.tensor(1/65))

tensor(4.1744)

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    # every once in a while evaluate the loss on train and val sets
    if epoch % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # Train
    _, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    break

step 0: train loss 4.3786, val loss 4.3828


In [15]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


EpyeYnrFiK?k,JCdLYy
fb!DMd,NY-aWoTbJt:
nGpvgLENkiKokvPSQ.!Cdg!uhhFfyK
FuPs,jnea'jbz'vA'l w pA'?NvH
occvCC?WHWqobfSOTVos;huRPfnX.X ;'lQuhhujk''HwzQQDNB;Z?V-RAyhzyl-a'Kd3RO:
lCpxov3RIpci SGL3LQhtqfhtYngZQ.uqLYYBOIqWimfgag!.Aw'BSjd,.OQ:QbYSz.;Z;bzQ:kF.r
GitmJj,3GldRnlvglqLA:IjBFOpv3qOoLXMiL$ tP !xG,bp,BVeT;znt
Em ,!YVxcKGkAqt3q$SXWTT& PQYtRhaqNvZyTkb
FaDprskwxZZP&IN;GGDXaa'S
NULYhwdJofdO ;.qWEKt ifKGhagvp$PQnzwoq
$BEoAUoX3hTJBUV-WR mKHOMmKfQ'!gel?SVYYcyTDlynCTCn-e-KqP!,hzsx$'.EFr&WXuz'3B'S?!, OjsV.


## Transformer

In [23]:
torch.manual_seed(1337)

B, T, C = 4, 8, 2 # batch, tokens, channels
x = torch.randn(B, T, C)

# single head
head_size = 6
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)


k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)

wei = q @ k.transpose(-2, -1) * head_size**0.5
                
tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros_like(tril)
wei = wei.masked_fill(tril==0, -torch.inf)
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v 

out.shape

torch.Size([4, 8, 2])

In [25]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3922, 0.6078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3374, 0.2906, 0.3720, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2354, 0.1856, 0.2737, 0.3052, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1688, 0.0986, 0.1922, 0.2080, 0.3323, 0.0000, 0.0000, 0.0000],
        [0.1408, 0.2323, 0.1108, 0.0937, 0.0906, 0.3317, 0.0000, 0.0000],
        [0.1435, 0.1496, 0.1466, 0.1493, 0.1297, 0.1349, 0.1463, 0.0000],
        [0.0903, 0.0607, 0.0865, 0.0821, 0.1863, 0.0979, 0.0803, 0.3158]],
       grad_fn=<SelectBackward0>)

In [26]:
out[0]

tensor([[ 0.1808, -0.0700],
        [-0.1477, -0.5837],
        [ 0.1893, -0.2800],
        [ 0.4385, -0.1597],
        [ 0.4339,  0.3043],
        [-0.3144, -0.2771],
        [ 0.1057, -0.0860],
        [-0.2098,  0.5815]], grad_fn=<SelectBackward0>)